# 02 RAG | 02 Embeddings | 01 Text Embeddings

## Azure Environment

Necessary parameter are imported from [./Configuration/application.env]. Check [Create Environment](../../01_CreateEnvironment/01_Environment.ipynb) to setup the necessary demo environment.


## Step 1: Create OpenAIClient / EmbeddingClient

The `AzureOpenAIClient` from the Azure.AI.OpenAI NuGet package offers a unified interface to create various specialized clients, each designed to handle specific tasks.
In this notebook the `EmbeddingClient` is used to create multi dimensional vectors from provided text.


In [1]:
#r "nuget: Azure.AI.OpenAI, 2.0.0-beta.2"
#r "nuget: DotNetEnv, 2.5.0"

using Azure; 
using Azure.AI.OpenAI;
using OpenAI.Embeddings;
using DotNetEnv; 
using System.IO;
using System.ClientModel;

//configuration file is created during environment creation
static string configurationFile = @"../../Configuration/application.env";
Env.Load(configurationFile);

string assetsFolder = Environment.GetEnvironmentVariable("WS_ASSETS_FOLDER") ?? "WS_ASSETS_FOLDER not found";;

string oAiApiKey = Environment.GetEnvironmentVariable("WS_AOAI_APIKEY") ?? "WS_AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("WS_AOAI_ENDPOINT") ?? "WS_AOAI_ENDPOINT not found";
string embeddingDeploymentName = Environment.GetEnvironmentVariable("WS_EMBEDDING_DEPLOYMENTNAME") ?? "WS_EMBEDDING_DEPLOYMENTNAME not found";

ApiKeyCredential apiKeyCredential = new ApiKeyCredential(oAiApiKey);
AzureOpenAIClient azureOpenAIClient = new AzureOpenAIClient(new Uri(oAiEndpoint), apiKeyCredential);

EmbeddingClient embeddingClient = azureOpenAIClient.GetEmbeddingClient(embeddingDeploymentName);

Console.WriteLine($"AzureOpenAI Client created...");
Console.WriteLine($"EmbeddingClient created...");

Installed Packages Azure.AI.OpenAI, 2.0.0-beta.2 DotNetEnv, 2.5.0

AzureOpenAI Client created...
EmbeddingClient created...


## Step 2: Create Embeddings

Embeddings will be created from the following text resources: 

- [WikiAKS.txt](../../Assets/Embedding/WikiAks.txt): A download from the MS Azure Kubernetes Service documentation
- [WikiSuperBowl2024](../../Assets/Embedding/WikiSuperBowl2024.txt): A download from Wikipedia with information about the 2024 Super Bowl
- The string literal: *Who won the Super Bowl in 2024* 
- The string literal: *The Kansas City Chiefs won the Super Bowl in 2024*

In [4]:

using System.ClientModel; 

// Vectorize AKS (Azure Kubernetes Service) information from Wikipedia
string documentationPage = Path.Combine(assetsFolder,"Embedding", "WikiAks.txt");
string textToBeVectorized = File.ReadAllText(documentationPage);

ClientResult<Embedding> clientResultAKS =  
    await embeddingClient.GenerateEmbeddingAsync(textToBeVectorized);
        
Console.WriteLine($"Vector wiki info AKS created...");

// Vectorize Super Bowl information from Wikipedia
documentationPage = Path.Combine(assetsFolder,"Embedding", "WikiSuperBowl2024.txt");
textToBeVectorized = File.ReadAllText(documentationPage);

ClientResult<Embedding> clientResultSuperBowl =  
    await embeddingClient.GenerateEmbeddingAsync(textToBeVectorized);

Console.WriteLine($"Vector wiki info Super Bowl created...");

// Vectorize a question
textToBeVectorized = "Who won the Super Bowl in 2024";
ClientResult<Embedding> clientResultQuestion = 
    await embeddingClient.GenerateEmbeddingAsync(textToBeVectorized);

Console.WriteLine($"Vector question Super Bowl created...");

// Vectorize a statement
textToBeVectorized = "The Kansas City Chiefs won the Super Bowl in 2024";
ClientResult<Embedding> clientStatementQuestion = 
    await embeddingClient.GenerateEmbeddingAsync(textToBeVectorized);

Console.WriteLine($"Vector statement Super Bowl created...");


Vector wiki info AKS created...
Vector wiki info Super Bowl created...
Vector question Super Bowl created...
Vector statement Super Bowl created...


## Step 3: Vector storage

All four created vectors are stored in a txt file for analyze and further usage in other notebooks.

In [5]:
using System.Linq;

string wikiAKSFileName = Path.Combine(assetsFolder, "Embedding", "TextEmbedding_WikiAks.txt");
string wikiSuperBowlFileName = Path.Combine(assetsFolder, "Embedding", "TextEmbedding_WikiSuperBowl.txt");
string queryVectorFileName = Path.Combine(assetsFolder, "Embedding", "TextEmbedding_Query.txt");
string statementVectorFileName = Path.Combine(assetsFolder, "Embedding", "TextEmbedding_Statement.txt");

await File.WriteAllLinesAsync(
    wikiAKSFileName, 
    clientResultAKS.Value.Vector.ToArray().Select(vector => vector.ToString())
);

await File.WriteAllLinesAsync(
    wikiSuperBowlFileName, 
    clientResultSuperBowl.Value.Vector.ToArray().Select(vector => vector.ToString())
);

await File.WriteAllLinesAsync(
    queryVectorFileName, 
    clientResultQuestion.Value.Vector.ToArray().Select(vector => vector.ToString())
);

await File.WriteAllLinesAsync(
    statementVectorFileName, 
    clientStatementQuestion.Value.Vector.ToArray().Select(vector => vector.ToString())
);


Console.WriteLine($"AKS vector stored in {wikiAKSFileName}...");
Console.WriteLine($"Super Bowl vector stored in {wikiSuperBowlFileName}...");
Console.WriteLine($"Query Vector stored in {queryVectorFileName}...");
Console.WriteLine($"Statement Vector stored in {statementVectorFileName}...");


AKS vector stored in ../../assets\Embedding\TextEmbedding_WikiAks.txt...
Super Bowl vector stored in ../../assets\Embedding\TextEmbedding_WikiSuperBowl.txt...
Query Vector stored in ../../assets\Embedding\TextEmbedding_Query.txt...
Statement Vector stored in ../../assets\Embedding\TextEmbedding_Statement.txt...
